In [1]:
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras.preprocessing.image import ImageDataGenerator
import utils as U
reload(U)
import image_loader as IM
reload(IM)
from theano.sandbox import cuda
from keras.preprocessing import image
%matplotlib inline
import os
from keras.models import Model
from keras.layers import Dense, BatchNormalization, Input, Flatten, Conv2D, AveragePooling2D
from keras.optimizers import Adam

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)


In [2]:
mapping = U.binary_one_hot_mapping("partly_cloudy")

In [3]:
klass = "partly_cloudy"
folder = "data/train/"
train_folder = folder+"train"
train_filenames = os.listdir(train_folder)
validation_folder = folder+"valid"
validation_filenames = os.listdir(validation_folder)
batch_size = 64
train_batch_count = len(train_filenames) / batch_size
val_batch_count = len(validation_filenames) / batch_size

# VGG

In [4]:
import models.model as M
from keras.layers import Flatten, Dense, BatchNormalization
from keras.models import Sequential
from keras.layers import Dropout
from keras.regularizers import l2
reload(M)

<module 'models.model' from 'models/model.pyc'>

In [15]:
model = M.full_model()
for l in model.layers:
    l.trainable = False

## Pre compute convolutional filters

In [16]:
gen_t = image.ImageDataGenerator(vertical_flip=True, horizontal_flip=True)

gen = IM.DirectoryIterator(train_folder, train_filenames, image_processor=gen_t, one_hot_filename_mapping=mapping, batch_size=batch_size, shuffle=True)
val_gen = IM.DirectoryIterator(validation_folder, validation_filenames, one_hot_filename_mapping=mapping, batch_size=batch_size, shuffle=False)

In [17]:
augmented_conv_features = conv_model.predict_generator(gen, train_batch_count, verbose=1)
augmented_val_features = conv_model.predict_generator(val_gen, val_batch_count, verbose=1)

NameError: name 'conv_model' is not defined

In [21]:
#U.save_array("precomputed-conv-features", augmented_conv_features)
#U.save_array("precomputed-conv-val-features", augmented_val_features)
augmented_conv_features = U.load_array("precomputed-conv-features")
augmented_val_features = U.load_array("precomputed-conv-val-features")

## Training Dense Layers, Tuning Dropout and L2 Regularization

In [22]:
from utils import binary_one_hot_mapping
mapping = binary_one_hot_mapping(klass)

train_labels = [ mapping[fname] for fname in train_filenames ]
valid_labels = [ mapping[fname] for fname in validation_filenames ]

### Question
    * Binary crossetropy on Dense(1) vs Categorical on Dense(2)?
    * Can we weight loss differently for different categories?

In [23]:
dense_layers = [
    Flatten(input_shape=(8,8,512)),
    BatchNormalization(),
    Dense(4096, activation="relu", kernel_regularizer=l2(0.005)),
    BatchNormalization(),
    Dropout(0.4),
    Dense(4096, activation="relu", kernel_regularizer=l2(0.005)),
    BatchNormalization(),
    Dropout(0.4),
    Dense(2, activation="softmax"),
]

dense_model = Sequential(dense_layers)
conv_model = model
#model = Sequential(model.layers + dense_layers)

In [26]:
dense_model.load_weights("weights/dense_weights_partly_cloudy_2")
dense_model.compile(optimizer=(Adam(lr=0.0001)), metrics=["accuracy"], loss="categorical_crossentropy")

In [ ]:
dense_model.fit(augmented_conv_features, train_labels, batch_size=batch_size, epochs=2, validation_data=(augmented_val_features, valid_labels))

In [9]:
dense_model.load_weights("weights/dense_weights_partly_cloudy")
#dense_model.optimizer.lr = 0.00001
#dense_model.fit(augmented_conv_features, train_labels, batch_size=batch_size, epochs=1, validation_data=(augmented_val_features, valid_labels))

In [ ]:
dense_model.save_weights("weights/dense_weights_partly_cloudy_2")
dense_model.optimizer.lr = 0.00001
dense_model.fit(augmented_conv_features, train_labels, batch_size=batch_size, epochs=2, validation_data=(augmented_val_features, valid_labels))

## Full Model

In [35]:
#dense_model.load_weights("weights/dense_weights_partly_cloudy_2")

dense_layers = [
    Flatten(input_shape=(8,8,512)),
    BatchNormalization(name="batch_norm_1"),
    Dense(4096, activation="relu", name="Dense 1"),
    BatchNormalization(name="batch_norm_2"),
    Dropout(0.0),
    Dense(4096, activation="relu", name="Dense 2"),
    BatchNormalization(name="batch_norm_3"),
    Dropout(0.0),
    Dense(2, activation="softmax", name="Dense Output Layer"),
]

m = Sequential(dense_layers)
m.load_weights("weights/dense_weights_partly_cloudy_2")

In [36]:
conv_layers = M.vgg().layers
full_model = Sequential(conv_layers + dense_layers)

full_model.save_weights("weights/partly_cloudy_full_model_top_trained")

In [ ]:
klass = "partly_cloudy"
folder = "data/samples/partly_cloudy/"
train_folder = folder+"train"
train_filenames = os.listdir(train_folder)
validation_folder = folder+"valid"
validation_filenames = os.listdir(validation_folder)
batch_size = 64
train_batch_count = len(train_filenames) / batch_size
val_batch_count = len(validation_filenames) / batch_size
gen = IM.DirectoryIterator(train_folder, train_filenames, one_hot_filename_mapping=mapping, batch_size=batch_size, shuffle=False)

for l in full_model.layers:
    l.trainable = False

full_model.compile(optimizer=(Adam(lr=0.0001)), metrics=["accuracy"], loss="categorical_crossentropy")
preds = full_model.fit_generator(gen, train_batch_count, validation_data=val_gen, validation_steps=val_batch_count)

res = zip(train_filenames, preds)

In [ ]:
#loss: 1.6267 - acc: 0.7344 - val_loss: 0.6926 - val_acc: 0.8424
wrong = U.incorrect_predictions(res, mapping)
print(res[0])
mapping['train_28750.jpg']
from matplotlib import pyplot as plt
img = IM.load_to_numpy("data/train-jpg/"+wrong[0][0])
gen_t = image.ImageDataGenerator(horizontal_flip=True, vertical_flip=True)
img = gen_t.random_transform(img)
plt.imshow(img)
#U.plot("data/train-jpg/"+wrong[0][0])


# Full Training

In [81]:
from keras.callbacks import LearningRateScheduler, ModelCheckpoint, BaseLogger, History, CSVLogger
callbacks = [
    History(),
    BaseLogger(),
    ModelCheckpoint("weights/partly-cloudy-full-training.{epoch:02d}-{val_loss:.2f}.hdf5", monitor="val_loss", verbose=0, save_best_only=False, save_weights_only=True, mode='auto', period=1),
    CSVLogger("results/partly_cloudy.csv", separator=',', append=False)
]

dense_layers = [
    Flatten(input_shape=(8,8,512)),
    BatchNormalization(),
    Dense(4096, activation="relu", kernel_regularizer=l2(0.005)),
    BatchNormalization(),
    Dropout(0.4),
    Dense(4096, activation="relu", kernel_regularizer=l2(0.005)),
    BatchNormalization(),
    Dropout(0.4),
    Dense(2, activation="softmax"),
]

In [83]:
conv_layers = M.vgg().layers
full_model = Sequential(conv_layers + dense_layers)
#full_model.load_weights("weights/partly_cloudy_full_model_top_trained")

In [39]:
batch_size = 32

klass = "partly_cloudy"
folder = "data/train/"
train_folder = folder+"train"
train_filenames = os.listdir(train_folder)
validation_folder = folder+"valid"
validation_filenames = os.listdir(validation_folder)

train_batch_count = len(train_filenames) / batch_size
val_batch_count = len(validation_filenames) / batch_size

gen_t = image.ImageDataGenerator(vertical_flip=True, horizontal_flip=True)

gen = IM.DirectoryIterator(train_folder, train_filenames, image_processor=gen_t, one_hot_filename_mapping=mapping, batch_size=batch_size, shuffle=True)
val_gen = IM.DirectoryIterator(validation_folder, validation_filenames, one_hot_filename_mapping=mapping, batch_size=batch_size, shuffle=False)

for l in full_model.layers:
    l.trainable = True

full_model.compile(optimizer=(Adam(lr=0.00001)), metrics=["accuracy"], loss="categorical_crossentropy")
full_model.fit_generator(gen, train_batch_count, epochs=2, validation_data=val_gen, validation_steps=val_batch_count , callbacks=callbacks)


Epoch 1/2
1088/1088 [==============================] - 5886s - loss: 0.6984 - acc: 0.9501 - val_loss: 0.5514 - val_acc: 0.9558
Epoch 2/2
1088/1088 [==============================] - 5886s - loss: 0.4640 - acc: 0.9611 - val_loss: 0.3912 - val_acc: 0.9641


In [40]:
full_model.fit_generator(gen, train_batch_count, epochs=2, validation_data=val_gen, validation_steps=val_batch_count , callbacks=callbacks)

Epoch 1/2
1088/1088 [==============================] - 5877s - loss: 0.3479 - acc: 0.9632 - val_loss: 0.3196 - val_acc: 0.9570
Epoch 2/2
1088/1088 [==============================] - 5868s - loss: 0.2808 - acc: 0.9641 - val_loss: 0.2531 - val_acc: 0.9675


In [ ]:
full_model.optimizer.lr = 0.000001
full_model.fit_generator(gen, train_batch_count, epochs=10, validation_data=val_gen, validation_steps=val_batch_count , callbacks=callbacks)